In [21]:
import setigen_patch as stg # some **kwargs, slightly modified plots
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u

In [22]:
%matplotlib inline

In [23]:
WINDOW_SIZE = 2**10

First I'll define MeerKAT band 5b parameters

In [29]:
lo, hi = 8.3*u.GHz, 15.4*u.GHz
center_freq = (lo+hi)/2
bandwidth = (hi-lo)
num_samples = 2**29
obs_time = 300 * u.s
n_ints = 16
df = bandwidth / num_samples
dt = obs_time / n_ints

dt, df.to(u.Hz)

(<Quantity 18.75 s>, <Quantity 13.22478056 Hz>)

In [30]:
frame = stg.Frame(
    fchans=num_samples,
    tchans=n_ints,
    df=df,
    dt=dt,
    fch1=hi,
)

MemoryError: Unable to allocate 64.0 GiB for an array with shape (8589934592,) and data type float64

In [ ]:
T_rx = 10.6 * u.K + 0.633 *(u.K/u.GHz) * (center_freq - 8.41 * u.GHz)
T_sky = 5 * u.K
T_sys = T_rx + T_sky

In [ ]:
noise = frame.add_noise(x_mean=0, x_std=np.sqrt(T_sys.value), noise_type='gaussian')

In [ ]:
frame.plot('f', minor_ticks=True, db=False, funits="GHz", fdownsample=2**8)

In [ ]:
def add_signal(frame, mean_freq, dr, snr):
    f_start = mean_freq - dr*frame.data.shape[0]*frame.dt*u.s/2
    
    return frame.add_signal(
        stg.constant_path(f_start=f_start,
                          drift_rate=dr),
        stg.constant_t_profile(level=frame.get_intensity(snr=snr)),
        stg.gaussian_f_profile(width=100*u.Hz),
        stg.constant_bp_profile(level=1)
    )

In [ ]:
signal = add_signal(frame, 12. * u.GHz, 1.5 * (u.Hz/u.s), 30)

In [ ]:
frame_s = frame.get_slice(
    (f := frame.get_index(frequency=12.*u.GHz)) - WINDOW_SIZE//2,
    f + WINDOW_SIZE//2
)

In [ ]:
frame_s.data.shape

In [ ]:
frame_s.plot(db=False)